# Trade Under Pressure



## Import packages

In [2]:
import pandas as pa

## Data preprocessing
### Dist CEPII

In [3]:
# Loading and preprocessing Dist CEPII dataset
dist_cepii = pa.read_excel("./data/geographic/dist_cepii.xls",
                           index_col=None, header=0,
                           na_values=["."], verbose=True, decimal=",",
                           dtype={"contig": bool, "comlang_off": bool,
                                  "comlang_ethno": bool, "colony": bool,
                                  "smctry": bool}
                           )
dist_cepii = dist_cepii.drop(["comcol", "curcol", "col45"], axis=1)
dist_cepii = dist_cepii.rename(columns={"iso_o": "origin", "iso_d": "destination"})

Reading sheet 0


Inspect DataFrame properties

In [4]:
dist_cepii.head(n=50)

,origin,destination,contig,comlang_off,comlang_ethno,colony,smctry,dist,distcap,distw,distwces
0,ABW,ABW,False,False,False,False,False,5.225315,5.225315,25.09354,23.04723
1,ABW,AFG,False,False,False,False,False,13257.810000,13257.810000,13168.22000,13166.37000
2,ABW,AGO,False,False,False,False,False,9516.913000,9516.913000,9587.31600,9584.19300
3,ABW,AIA,False,False,True,False,False,983.268200,983.268200,976.89740,976.89160
4,ABW,ALB,False,False,False,False,False,9091.742000,9091.742000,9091.57600,9091.46600
5,ABW,AND,False,True,False,False,False,7572.788000,7572.788000,7570.08400,7570.08300
6,ABW,ANT,False,True,True,False,True,136.384800,136.384800,239.90640,142.85830
7,ABW,ARE,False,False,False,False,False,12735.010000,12735.010000,12773.08000,12772.95000
8,ABW,ARG,False,True,False,False,False,5396.220000,5396.220000,5187.78800,5157.12600
9,ABW,ARM,False,False,False,False,False,11107.780000,11107.780000,11106.96000,11106.76000


In [5]:
dist_cepii.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50176 entries, 0 to 50175
Data columns (total 11 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   origin         50176 non-null  object 
 1   destination    50176 non-null  object 
 2   contig         50176 non-null  bool   
 3   comlang_off    50176 non-null  bool   
 4   comlang_ethno  50176 non-null  bool   
 5   colony         50176 non-null  bool   
 6   smctry         50176 non-null  bool   
 7   dist           50176 non-null  float64
 8   distcap        50176 non-null  float64
 9   distw          47961 non-null  float64
 10  distwces       47961 non-null  float64
dtypes: bool(5), float64(4), object(2)
memory usage: 2.5+ MB


In [6]:
dist_cepii.dtypes

origin            object
destination       object
contig              bool
comlang_off         bool
comlang_ethno       bool
colony              bool
smctry              bool
dist             float64
distcap          float64
distw            float64
distwces         float64
dtype: object

Check how many rows have NaN

In [7]:
dist_cepii.isna().sum()

origin              0
destination         0
contig              0
comlang_off         0
comlang_ethno       0
colony              0
smctry              0
dist                0
distcap             0
distw            2215
distwces         2215
dtype: int64

Count how many times a country has NaN in Origin and in Destination

In [8]:
na_rows = dist_cepii[dist_cepii.isna().any(axis=1)][["origin", "destination"]]
# Sanity check
na_rows.isna().sum()

origin         0
destination    0
dtype: int64

In [9]:
na_rows[["origin"]].value_counts()

origin
PCN       224
CCK       224
MAC       224
MSR       224
CXR       224
         ... 
GNQ         5
GRC         5
GRD         5
GRL         5
ZWE         5
Name: count, Length: 224, dtype: int64

Check how much % of rows are NaN from the total amount

In [10]:
print("Percentage of NaN rows: ", (na_rows.shape[0]/dist_cepii.shape[0])*100, "%")

Percentage of NaN rows:  4.414461096938775 %


Drop NaN values

In [11]:
dist_cepii = dist_cepii.dropna()

In [12]:
# Sanity check
dist_cepii.isna().any()

origin           False
destination      False
contig           False
comlang_off      False
comlang_ethno    False
colony           False
smctry           False
dist             False
distcap          False
distw            False
distwces         False
dtype: bool

Check how many unique countries there are in both origin and destination. Numbers should match.

In [13]:
print("Unique countries in origin column", dist_cepii["origin"].nunique())
print("Unique countries in destination column", dist_cepii["destination"].nunique())

Unique countries in origin column 219
Unique countries in destination column 219


### GSDB V3 Dyadic

Inspect the `GSDB_V3_Dyadic.dta` - this should be a Stata file. What is the difference to `GSDB_V3.xls`?

In [14]:
gsdb_stata = pa.read_stata("data/sanctions/GSDB_V3_Dyadic.dta")
gsdb_xls = pa.read_excel("data/sanctions/GSDB_V3.xls")

Retrieve rows that include `case_id` 471

In [15]:
gsdb_stata[gsdb_stata["case_id"].astype(str).str.contains("471")]

,case_id,sanctioning_state_iso3,sanctioning_state,sanctioned_state_iso3,sanctioned_state,year,arms,military,trade,descr_trade,financial,travel,other,target_mult,sender_mult,objective,success
0,471,AFG,Afghanistan,AGO,Angola,1993-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
1,471,AFG,Afghanistan,AGO,Angola,1994-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
2,471,AFG,Afghanistan,AGO,Angola,1995-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
3,471,AFG,Afghanistan,AGO,Angola,1996-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
4,"471,573",AFG,Afghanistan,AGO,Angola,1997-01-01,1,1,1,exp_part,0,1,1,0,1,"end_war,end_war","success_total,success_total"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
150386,"471,594,573",ZWE,Zimbabwe,AGO,Angola,1998-01-01,1,1,1,"exp_part,imp_part",1,1,1,0,1,"end_war,end_war,end_war","success_total,success_total,success_total"
150387,"573,471,594",ZWE,Zimbabwe,AGO,Angola,1999-01-01,1,1,1,"exp_part,imp_part",1,1,1,0,1,"end_war,end_war,end_war","success_total,success_total,success_total"
150388,"471,594,573",ZWE,Zimbabwe,AGO,Angola,2000-01-01,1,1,1,"exp_part,imp_part",1,1,1,0,1,"end_war,end_war,end_war","success_total,success_total,success_total"
150389,"594,471,573",ZWE,Zimbabwe,AGO,Angola,2001-01-01,1,1,1,"exp_part,imp_part",1,1,1,0,1,"end_war,end_war,end_war","success_total,success_total,success_total"


In [16]:
gsdb_stata.head(500)

,case_id,sanctioning_state_iso3,sanctioning_state,sanctioned_state_iso3,sanctioned_state,year,arms,military,trade,descr_trade,financial,travel,other,target_mult,sender_mult,objective,success
0,471,AFG,Afghanistan,AGO,Angola,1993-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
1,471,AFG,Afghanistan,AGO,Angola,1994-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
2,471,AFG,Afghanistan,AGO,Angola,1995-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
3,471,AFG,Afghanistan,AGO,Angola,1996-01-01,1,1,1,exp_part,0,0,0,0,1,end_war,success_total
4,"471,573",AFG,Afghanistan,AGO,Angola,1997-01-01,1,1,1,exp_part,0,1,1,0,1,"end_war,end_war","success_total,success_total"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,"738,869,754",AFG,Afghanistan,SDN,Sudan,2010-01-01,1,1,0,,1,1,0,0,1,"human_rights,end_war,human_rights,end_war,huma...","ongoing,ongoing,ongoing,ongoing,ongoing,ongoing"
496,"738,754,869",AFG,Afghanistan,SDN,Sudan,2011-01-01,1,1,0,,1,1,0,0,1,"human_rights,end_war,human_rights,end_war,huma...","ongoing,ongoing,ongoing,ongoing,ongoing,ongoing"
497,"754,738,869",AFG,Afghanistan,SDN,Sudan,2012-01-01,1,1,0,,1,1,0,0,1,"human_rights,end_war,human_rights,end_war,huma...","ongoing,ongoing,ongoing,ongoing,ongoing,ongoing"
498,"869,754,738",AFG,Afghanistan,SDN,Sudan,2013-01-01,1,1,0,,1,1,0,0,1,"human_rights,end_war,human_rights,end_war,huma...","ongoing,ongoing,ongoing,ongoing,ongoing,ongoing"


In [17]:
gsdb_xls.head(500)

,case_id,sanctioned_state,sanctioning_state,begin,end,trade,descr_trade,arms,military,financial,travel,other,target_mult,sender_mult,objective,success
0,1,German Democratic Republic,Germany,1949,1973,0,NaN,0,0,0,0,1,0,0,territorial_conflict,success_total
1,2,Pakistan,India,1949,1951,1,"exp_compl, imp_compl",0,0,0,0,0,0,0,policy_change,nego_settlement
2,3,Bulgaria,United States,1950,1966,0,NaN,0,0,0,0,1,0,0,destab_regime,failed
3,4,Bulgaria,United States,1950,1959,0,NaN,0,0,0,1,0,0,0,destab_regime,success_part
4,5,Bulgaria,United States,1950,1963,0,NaN,0,0,1,0,0,0,0,destab_regime,success_part
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,496,Russia,Ukraine,1993,1996,1,imp_part,0,1,0,0,0,0,0,policy_change,nego_settlement
496,497,Sudan,United States,1993,2020,0,NaN,0,0,1,0,0,0,0,terrorism,success_part
497,498,Togo,EU,1993,2007,0,NaN,0,0,1,0,0,0,1,"democracy,human_rights","success_total,success_total"
498,499,Togo,France,1993,2008,0,NaN,0,0,1,0,0,0,0,"human_rights,democracy,end_war","success_total,success_total,success_total"


In [18]:
print("Shape for Stata data: ", gsdb_stata.shape)
print("Shape for XLS data:", gsdb_xls.shape)

Shape for Stata data:  (150875, 17)
Shape for XLS data: (1325, 16)


In [19]:
# Column names Stata
gsdb_stata.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150875 entries, 0 to 150874
Data columns (total 17 columns):
 #   Column                  Non-Null Count   Dtype         
---  ------                  --------------   -----         
 0   case_id                 150875 non-null  object        
 1   sanctioning_state_iso3  150875 non-null  object        
 2   sanctioning_state       150875 non-null  object        
 3   sanctioned_state_iso3   150875 non-null  object        
 4   sanctioned_state        150875 non-null  object        
 5   year                    150875 non-null  datetime64[ns]
 6   arms                    150875 non-null  int8          
 7   military                150875 non-null  int8          
 8   trade                   150875 non-null  int8          
 9   descr_trade             150875 non-null  object        
 10  financial               150875 non-null  int8          
 11  travel                  150875 non-null  int8          
 12  other                   150875

In [20]:
gsdb_stata.dtypes

case_id                           object
sanctioning_state_iso3            object
sanctioning_state                 object
sanctioned_state_iso3             object
sanctioned_state                  object
year                      datetime64[ns]
arms                                int8
military                            int8
trade                               int8
descr_trade                       object
financial                           int8
travel                              int8
other                               int8
target_mult                         int8
sender_mult                         int8
objective                         object
success                           object
dtype: object

In [21]:
# Column names XLS
gsdb_xls.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1325 entries, 0 to 1324
Data columns (total 16 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   case_id            1325 non-null   int64 
 1   sanctioned_state   1325 non-null   object
 2   sanctioning_state  1325 non-null   object
 3   begin              1325 non-null   int64 
 4   end                1325 non-null   int64 
 5   trade              1325 non-null   int64 
 6   descr_trade        513 non-null    object
 7   arms               1325 non-null   int64 
 8   military           1325 non-null   int64 
 9   financial          1325 non-null   int64 
 10  travel             1325 non-null   int64 
 11  other              1325 non-null   int64 
 12  target_mult        1325 non-null   int64 
 13  sender_mult        1325 non-null   int64 
 14  objective          1325 non-null   object
 15  success            1325 non-null   object
dtypes: int64(11), object(5)
memory usage: 165.

In [22]:
gsdb_xls.dtypes

case_id               int64
sanctioned_state     object
sanctioning_state    object
begin                 int64
end                   int64
trade                 int64
descr_trade          object
arms                  int64
military              int64
financial             int64
travel                int64
other                 int64
target_mult           int64
sender_mult           int64
objective            object
success              object
dtype: object

## GDP Data

In [23]:
gdp = pa.read_csv(filepath_or_buffer="data/economic/GDP(currentUSD)_1974-2023.csv", sep=",", na_values=[".."])

In [24]:
gdp = gdp.drop(["Series Name", "Series Code", "Country Name"], axis=1)

In [25]:
gdp = gdp.rename(columns=lambda x: x if not x.endswith("]") else x.split(" ")[0])

In [26]:
gdp.head(10)

,Country Code,1974,1975,1976,1977,1978,1979,1980,1981,1982,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,AFG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.049713e+10,1.913422e+10,1.811657e+10,1.875346e+10,1.805322e+10,1.879944e+10,1.995593e+10,1.426000e+10,1.449724e+10,1.723305e+10
1,ALB,NaN,NaN,NaN,NaN,NaN,NaN,1.578102e+09,1.808177e+09,1.861163e+09,...,1.322815e+10,1.138685e+10,1.186120e+10,1.301973e+10,1.537951e+10,1.558511e+10,1.524146e+10,1.803201e+10,1.901724e+10,2.354718e+10
2,DZA,1.320987e+10,1.555790e+10,1.772824e+10,2.097211e+10,2.636449e+10,3.324371e+10,4.234583e+10,4.434859e+10,4.520717e+10,...,2.389427e+11,1.874939e+11,1.807638e+11,1.898809e+11,1.945545e+11,1.934597e+11,1.648734e+11,1.862312e+11,2.256385e+11,2.476262e+11
3,ASM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6.430000e+08,6.730000e+08,6.710000e+08,6.120000e+08,6.390000e+08,6.470000e+08,7.210000e+08,7.500000e+08,8.710000e+08,NaN
4,AND,1.865571e+08,2.201126e+08,2.272839e+08,2.539979e+08,3.080203e+08,4.115487e+08,4.463778e+08,3.889833e+08,3.759147e+08,...,3.271686e+09,2.789881e+09,2.896610e+09,3.000162e+09,3.218420e+09,3.155149e+09,2.891001e+09,3.324648e+09,3.380613e+09,3.785067e+09
5,AGO,NaN,NaN,NaN,NaN,NaN,NaN,5.930503e+09,5.550483e+09,5.550483e+09,...,1.359668e+11,9.049642e+10,5.276162e+10,7.369015e+10,7.945069e+10,7.089796e+10,4.850156e+10,6.650513e+10,1.043997e+11,8.482465e+10
6,ATG,NaN,NaN,NaN,7.749630e+07,8.803333e+07,1.095852e+08,1.324407e+08,1.493778e+08,1.664259e+08,...,1.378830e+09,1.437756e+09,1.489693e+09,1.531152e+09,1.661530e+09,1.725352e+09,1.410796e+09,1.601367e+09,1.867733e+09,2.033085e+09
7,ARG,7.243678e+10,5.243865e+10,5.116950e+10,5.678100e+10,8.904945e+10,6.925233e+10,7.696192e+10,7.867684e+10,8.430749e+10,...,5.263197e+11,5.947493e+11,5.575323e+11,6.436284e+11,5.248199e+11,4.477547e+11,3.857405e+11,4.865641e+11,6.327901e+11,6.460753e+11
8,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,1.160951e+10,1.055334e+10,1.054614e+10,1.152746e+10,1.245794e+10,1.361929e+10,1.264170e+10,1.387891e+10,1.951351e+10,2.408575e+10
9,ABW,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2.790850e+09,2.962907e+09,2.983635e+09,3.092429e+09,3.276184e+09,3.395799e+09,2.481857e+09,2.929447e+09,3.279344e+09,3.648573e+09


In [27]:
gdp.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 222 entries, 0 to 221
Data columns (total 51 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   Country Code  217 non-null    object 
 1   1974          147 non-null    float64
 2   1975          150 non-null    float64
 3   1976          150 non-null    float64
 4   1977          153 non-null    float64
 5   1978          152 non-null    float64
 6   1979          153 non-null    float64
 7   1980          163 non-null    float64
 8   1981          163 non-null    float64
 9   1982          164 non-null    float64
 10  1983          164 non-null    float64
 11  1984          166 non-null    float64
 12  1985          168 non-null    float64
 13  1986          168 non-null    float64
 14  1987          174 non-null    float64
 15  1988          176 non-null    float64
 16  1989          176 non-null    float64
 17  1990          192 non-null    float64
 18  1991          193 non-null    

In [28]:
gdp.shape

(222, 51)

Check how many countries there are in the dataset.

In [29]:
gdp["Country Code"].nunique()

217

In [30]:
gdp["Country Code"].isna().sum()

np.int64(5)

Check countries that are in CEPII, but not in GDP dataset.

In [31]:
countries_unique_to_cepii = list(set(dist_cepii["origin"].unique()) - set(gdp["Country Code"].unique()))
countries_unique_to_cepii

['WLF',
 'NFK',
 'GUF',
 'ANT',
 'SHN',
 'COK',
 'REU',
 'AIA',
 'PAL',
 'TMP',
 'NIU',
 'TWN',
 'GLP',
 'TKL',
 'MTQ',
 'ROM',
 'SPM',
 'ZAR',
 'ESH',
 'FLK',
 'YUG']

Check countries that are in GDP, but not in CEPII dataset.

In [32]:
countries_unique_to_gdp = list(set(gdp["Country Code"].unique()) - set(dist_cepii["origin"].unique()))
countries_unique_to_gdp

[nan,
 'ROU',
 'PSE',
 'CUW',
 'MNE',
 'SSD',
 'IMN',
 'SRB',
 'GUM',
 'COD',
 'XKX',
 'TLS',
 'SXM',
 'MCO',
 'ASM',
 'CHI',
 'MAF',
 'VIR',
 'MAC',
 'LIE']

#### --- ENDE GELÄNDE ---